In [1]:
%%shell
pip install apache-tvm --pre

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 MB 7.6 MB/s eta 0:00:00


In [2]:
!tvmc --help

usage: tvmc [--config CONFIG] [-v] [--version] [-h] {micro,run,tune,compile} ...

TVM compiler driver

options:
  --config CONFIG       configuration json file
  -v, --verbose         increase verbosity
  --version             print the version and exit
  -h, --help            show this help message and exit.

commands:
  {micro,run,tune,compile}
    micro               select micro context.
    run                 run a compiled module
    tune                auto-tune a model
    compile             compile a model.

TVMC - TVM driver command-line interface


In [3]:
!pip3 install onnx onnxoptimizer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 678.1/678.1 kB 36.6 MB/s eta 0:00:00


In [4]:
!wget https://github.com/onnx/models/raw/b9a54e89508f101a1611cd64f4ef56b9cb62c7cf/vision/classification/resnet/model/resnet50-v2-7.onnx

--2024-09-06 04:02:41--  https://github.com/onnx/models/raw/b9a54e89508f101a1611cd64f4ef56b9cb62c7cf/vision/classification/resnet/model/resnet50-v2-7.onnx
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/onnx/models/b9a54e89508f101a1611cd64f4ef56b9cb62c7cf/vision/classification/resnet/model/resnet50-v2-7.onnx [following]
--2024-09-06 04:02:41--  https://media.githubusercontent.com/media/onnx/models/b9a54e89508f101a1611cd64f4ef56b9cb62c7cf/vision/classification/resnet/model/resnet50-v2-7.onnx
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102442450 (98M) [application/octet-stream]

In [5]:
import subprocess

subprocess.run([
    "tvmc", "compile",
    "--target", "llvm",
    "--input-shapes", "data:[1,3,224,224]",
    "--output", "resnet50-v2-7-tvm.tar",
    "resnet50-v2-7.onnx"
])

CompletedProcess(args=['tvmc', 'compile', '--target', 'llvm', '--input-shapes', 'data:[1,3,224,224]', '--output', 'resnet50-v2-7-tvm.tar', 'resnet50-v2-7.onnx'], returncode=0)

In [6]:
!mkdir model
!tar -xvf resnet50-v2-7-tvm.tar -C model
!ls model

mod.so
mod.json
mod.params
mod.json  mod.params  mod.so


In [7]:
from tvm.contrib.download import download_testdata
from PIL import Image
import numpy as np

img_url = "https://s3.amazonaws.com/model-server/inputs/kitten.jpg"
img_path = download_testdata(img_url, "imagenet_cat.png", module="data")

# Resize it to 224x224
resized_image = Image.open(img_path).resize((224, 224))
img_data = np.asarray(resized_image).astype("float32")

# ONNX expects NCHW input, so convert the array
img_data = np.transpose(img_data, (2, 0, 1))

# Normalize according to ImageNet
imagenet_mean = np.array([0.485, 0.456, 0.406])
imagenet_stddev = np.array([0.229, 0.224, 0.225])
norm_img_data = np.zeros(img_data.shape).astype("float32")
for i in range(img_data.shape[0]):
      norm_img_data[i, :, :] = (img_data[i, :, :] / 255 - imagenet_mean[i]) / imagenet_stddev[i]

# Add batch dimension
img_data = np.expand_dims(norm_img_data, axis=0)

# Save to .npz (outputs imagenet_cat.npz)
np.savez("imagenet_cat", data=img_data)

In [8]:
!tvmc run \
--inputs imagenet_cat.npz \
--output predictions.npz \
resnet50-v2-7-tvm.tar

2024-09-06 04:05:39.568 INFO load_module /tmp/tmp7dg1581a/mod.so


In [9]:
import os.path
import numpy as np

from scipy.special import softmax

from tvm.contrib.download import download_testdata

# Download a list of labels
labels_url = "https://s3.amazonaws.com/onnx-model-zoo/synset.txt"
labels_path = download_testdata(labels_url, "synset.txt", module="data")

with open(labels_path, "r") as f:
    labels = [l.rstrip() for l in f]

output_file = "predictions.npz"

# Open the output and read the output tensor
if os.path.exists(output_file):
    with np.load(output_file) as data:
        scores = softmax(data["output_0"])
        scores = np.squeeze(scores)
        ranks = np.argsort(scores)[::-1]

        for rank in ranks[0:5]:
            print("class='%s' with probability=%f" % (labels[rank], scores[rank]))

class='n02123045 tabby, tabby cat' with probability=0.621104
class='n02123159 tiger cat' with probability=0.356378
class='n02124075 Egyptian cat' with probability=0.019712
class='n02129604 tiger, Panthera tigris' with probability=0.001215
class='n04040759 radiator' with probability=0.000262


In [10]:
!pip install xgboost

In [11]:
!tvmc tune \
--target "llvm" \
--output resnet50-v2-7-autotuner_records.json \
resnet50-v2-7.onnx

[Task  1/25]  Current/Best:   13.50/  16.55 GFLOPS | Progress: (40/40) | 69.49 s Done.
[Task  2/25]  Current/Best:    3.10/  18.64 GFLOPS | Progress: (40/40) | 40.50 s Done.
[Task  3/25]  Current/Best:    8.75/  18.17 GFLOPS | Progress: (40/40) | 50.94 s Done.
[Task  4/25]  Current/Best:   13.72/  15.42 GFLOPS | Progress: (40/40) | 48.28 s Done.
[Task  5/25]  Current/Best:   18.65/  18.65 GFLOPS | Progress: (40/40) | 47.76 s Done.
[Task  6/25]  Current/Best:    5.08/  15.41 GFLOPS | Progress: (40/40) | 60.52 s Done.
[Task  7/25]  Current/Best:   17.71/  17.93 GFLOPS | Progress: (40/40) | 55.16 s Done.
[Task 10/25]  Current/Best:   10.18/  18.41 GFLOPS | Progress: (40/40) | 50.66 s Done.
[Task 11/25]  Current/Best:    5.00/  19.99 GFLOPS | Progress: (40/40) | 53.75 s Done.
[Task 13/25]  Current/Best:    7.90/  18.86 GFLOPS | Progress: (40/40) | 61.96 s Done.
[Task 14/25]  Current/Best:   11.64/  13.08 GFLOPS | Progress: (22/40) | 54.27 s Done.
 Done.
[Task 16/25]  Current/Best:    4.30/

In [12]:
!tvmc tune \
--target "llvm -mcpu=broadwell" \
--output resnet50-v2-7-autotuner_records.json \
resnet50-v2-7.onnx

[Task  1/25]  Current/Best:   29.53/  38.21 GFLOPS | Progress: (40/40) | 64.68 s Done.
[Task  2/25]  Current/Best:    7.25/  57.57 GFLOPS | Progress: (40/40) | 40.50 s Done.
[Task  3/25]  Current/Best:   22.70/  65.28 GFLOPS | Progress: (40/40) | 42.98 s Done.
[Task  4/25]  Current/Best:   16.97/  24.28 GFLOPS | Progress: (40/40) | 56.73 s Done.
[Task  5/25]  Current/Best:   10.87/  35.04 GFLOPS | Progress: (40/40) | 44.97 s Done.
[Task  6/25]  Current/Best:    5.48/  31.67 GFLOPS | Progress: (40/40) | 54.05 s Done.
[Task  7/25]  Current/Best:   10.12/  38.01 GFLOPS | Progress: (40/40) | 41.99 s Done.
[Task  8/25]  Current/Best:   14.54/  33.12 GFLOPS | Progress: (40/40) | 62.74 s Done.
[Task  9/25]  Current/Best:   12.06/  38.02 GFLOPS | Progress: (40/40) | 63.18 s Done.
[Task 10/25]  Current/Best:   24.41/  59.50 GFLOPS | Progress: (40/40) | 33.13 s Done.
[Task 11/25]  Current/Best:   34.65/  55.63 GFLOPS | Progress: (40/40) | 44.56 s Done.
[Task 12/25]  Current/Best:   17.37/  30.50

In [13]:
!tvmc compile \
--target "llvm" \
--tuning-records resnet50-v2-7-autotuner_records.json  \
--output resnet50-v2-7-tvm_autotuned.tar \
resnet50-v2-7.onnx

In [15]:
!tvmc run \
--inputs imagenet_cat.npz \
--output predictions.npz \
resnet50-v2-7-tvm_autotuned.tar

2024-09-06 05:54:32.654 INFO load_module /tmp/tmpisi5edo7/mod.so


In [20]:
!python postprocess.py

class='n02123045 tabby, tabby cat' with probability=0.621103
class='n02123159 tiger cat' with probability=0.356379
class='n02124075 Egyptian cat' with probability=0.019712
class='n02129604 tiger, Panthera tigris' with probability=0.001215
class='n04040759 radiator' with probability=0.000262


In [21]:
!tvmc run \
--inputs imagenet_cat.npz \
--output predictions.npz  \
--print-time \
--repeat 100 \
resnet50-v2-7-tvm_autotuned.tar

2024-09-06 06:24:09.272 INFO load_module /tmp/tmpu7sos5he/mod.so
Execution time summary:
 mean (ms)   median (ms)    max (ms)     min (ms)     std (ms)  
  603.0624     526.0517     994.8410     505.1502     163.2692                 


In [23]:
!tvmc run \
--inputs imagenet_cat.npz \
--output predictions.npz  \
--print-time \
--repeat 100 \
resnet50-v2-7-tvm.tar

2024-09-06 06:26:27.139 INFO load_module /tmp/tmpbvpve3d0/mod.so
Execution time summary:
 mean (ms)   median (ms)    max (ms)     min (ms)     std (ms)  
  749.7609     615.9257    1263.1543     595.5764     235.3068                 
